In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
import glob
import datetime as dt
%matplotlib inline

In [2]:
files = glob.glob("*.csv")
print(files)

#but some datasets have different timeframes

['particulate_matter.csv', 'ozone.csv', 'black_carbon.csv', 'nitric_oxide.csv', 'sulfur_dioxide.csv', 'nitrogen_dioxide.csv', 'black_carbon_mean.csv']


black carbon - summer 2012 to summer 2018
nitric oxide - summer 2013 to summer 2019
nitrogen dioxide - summer 2013 to summer 2019
particulate matter - summer 2013 to summer 2019
sulfur dioxide - winter 2008 to winter 2014
ozone - summer 2009 to summer 2019

In [3]:
#reframe the data to extract only mean values for each time interval per community district

cols = [0,3,4,6,7,8]
pols = {'Black Carbon':'BC',
        'Nitric Oxide (NO)':'NO',
        'Nitrogen Dioxide (NO2)':'NO2',
        'Ozone (O3)':'O3',
        'Fine Particulate Matter (PM2.5)':'PM2.5',
        'Sulfur Dioxide (SO2)':'SO2'}

df = pd.read_csv('black_carbon.csv')
#extract only CD data
df = df[df.GeoTypeName == 'Neighborhood (Community District)'].reset_index(drop=True)

df = df.iloc[:,cols].rename(columns={df.columns[3]:'CDname',
                                     df.columns[4]:'CD',
                                     df.columns[8]:'pol_mean'})

#split 'Summer 2018' into 'Summer' and '2018'
new = df['Year'].str.split(" ", n=1, expand=True) 
df['Year'] = new[1]
df['Month'] = new[0]
df['Day'] = 1

#convert Summer rows into month==5, Winter rows into month==11
for i in range(len(df)):
    if df.iloc[i,6] == 'Summer':
        df.iloc[i,6] = 5
    else:
        df.iloc[i,6] = 11
    
    df.iloc[i,1] = re.sub("[\(\[].*?[\)\]]", "", df.iloc[i,1]) #regex to remove '-'
    
    if '-' in df.iloc[i,0]:
        df.iloc[i,0] = df.iloc[i,0][:4]
    
df2 = df[['Year','Month','Day']]
df['Date'] = pd.to_datetime(df2).dt.date

df = df.iloc[:,[8,1,2,5]]
df = df.pivot_table(values='pol_mean', index='CD', columns='Date')
bc = df.reset_index()

bc.head()

Date   CD  2012-05-01  2012-11-01  2013-05-01  2013-11-01  2014-05-01  \
0     101         1.4         1.2         1.4         1.3         1.4   
1     102         1.7         1.7         1.9         1.7         1.9   
2     103         1.5         1.4         1.6         1.4         1.6   
3     104         1.6         1.6         1.7         1.6         1.7   
4     105         1.8         1.8         1.9         1.9         1.9   

Date  2014-11-01  2015-05-01  2015-11-01  2016-05-01  2016-11-01  2017-05-01  \
0            1.2         1.3         1.1         1.2         0.9         1.1   
1            1.5         1.7         1.5         1.7         1.1         1.4   
2            1.3         1.4         1.3         1.4         1.0         1.2   
3            1.4         1.6         1.4         1.5         1.1         1.3   
4            1.6         1.8         1.6         1.8         1.1         1.4   

Date  2017-11-01  2018-05-01  
0            1.1         1.3  
1            1.1         1.3  
2            1.0         1.1  
3            1.2         1.4  
4            1.4         1.7

In [7]:
bc.describe()

Date           CD  2012-05-01  2012-11-01  2013-05-01  2013-11-01  2014-05-01  \
count   59.000000   59.000000   59.000000   59.000000     59.0000   59.000000   
mean   280.305085    1.127119    1.130508    1.118644      1.1000    1.055932   
std    118.769010    0.237683    0.306400    0.289753      0.3085    0.318548   
min    101.000000    0.700000    0.700000    0.600000      0.5000    0.500000   
25%    203.500000    1.000000    0.900000    0.900000      0.8500    0.800000   
50%    306.000000    1.100000    1.100000    1.100000      1.1000    1.000000   
75%    402.500000    1.300000    1.400000    1.300000      1.3500    1.300000   
max    503.000000    1.800000    1.800000    1.900000      1.9000    1.900000   

Date   2014-11-01  2015-05-01  2015-11-01  2016-05-01  2016-11-01  2017-05-01  \
count   59.000000   59.000000   59.000000   59.000000   59.000000   59.000000   
mean     0.954237    1.033898    0.940678    0.979661    0.820339    0.967797   
std      0.247972    0.272036    0.219811    0.275293    0.163770    0.206315   
min      0.500000    0.600000    0.600000    0.500000    0.500000    0.500000   
25%      0.800000    0.800000    0.800000    0.800000    0.700000    0.800000   
50%      0.900000    1.000000    0.900000    0.900000    0.800000    1.000000   
75%      1.100000    1.200000    1.100000    1.100000    0.900000    1.100000   
max      1.600000    1.800000    1.600000    1.800000    1.100000    1.400000   

Date   2017-11-01  2018-05-01  
count   59.000000   59.000000  
mean     0.850847    0.888136  
std      0.170570    0.220567  
min      0.500000    0.500000  
25%      0.700000    0.750000  
50%      0.800000    0.900000  
75%      0.900000    1.000000  
max      1.400000    1.700000

In [55]:
bc.to_csv('black_carbon_mean.csv')

In [22]:
cols = [0,3,4,6,7,8]
pols = {'Black Carbon':'BC',
        'Nitric Oxide (NO)':'NO',
        'Nitrogen Dioxide (NO2)':'NO2',
        'Ozone (O3)':'O3',
        'Fine Particulate Matter (PM2.5)':'PM2.5',
        'Sulfur Dioxide (SO2)':'SO2'}

df = pd.read_csv('black_carbon.csv')
df = df[df.GeoTypeName == 'Neighborhood (Community District)'].reset_index(drop=True)

pol = df.iloc[0,5]
df = df.iloc[:,cols].rename(columns={df.columns[3]:'CDname',
                                     df.columns[4]:'CD',
                                     df.columns[6]:'{}-perc10'.format(pols[pol]),
                                     df.columns[7]:'{}-perc90'.format(pols[pol]),
                                     df.columns[8]:'{}-mean'.format(pols[pol])})

new = df['Year'].str.split(" ", n=1, expand=True) 
df['Year'] = new[1]
df['Month'] = new[0]
df['Day'] = 1

for i in range(len(df)):
    if df.iloc[i,6] == 'Summer':
        df.iloc[i,6] = 5
    else:
        df.iloc[i,6] = 11
    
    df.iloc[i,1] = re.sub("[\(\[].*?[\)\]]", "", df.iloc[i,1])
    
    if '-' in df.iloc[i,0]:
        df.iloc[i,0] = df.iloc[i,0][:4]
    
df2 = df[['Year','Month','Day']]
df['Date'] = pd.to_datetime(df2)

df = df.iloc[:,[8,1,2,3,4,5]]
maindf = df
maindf.head()

Date                         CDname   CD  BC-perc10  BC-perc90  \
0 2012-05-01   Bay Ridge and Dyker Heights   310        0.9        1.1   
1 2013-05-01   Bay Ridge and Dyker Heights   310        0.8        1.1   
2 2014-05-01   Bay Ridge and Dyker Heights   310        0.7        1.0   
3 2015-05-01   Bay Ridge and Dyker Heights   310        0.7        1.0   
4 2016-05-01   Bay Ridge and Dyker Heights   310        0.7        1.0   

   BC-mean  
0      1.0  
1      0.9  
2      0.9  
3      0.9  
4      0.8

In [9]:
cols = [0,3,4,6,7,8]
pols = {'Black Carbon':'BC',
        'Nitric Oxide (NO)':'NO',
        'Nitrogen Dioxide (NO2)':'NO2',
        'Ozone (O3)':'O3',
        'Fine Particulate Matter (PM2.5)':'PM2.5',
        'Sulfur Dioxide (SO2)':'SO2'}

for file in files:
    
    if file != 'black_carbon.csv':
        df = pd.read_csv(file)
        df = df[df.GeoTypeName == 'Neighborhood (Community District)'].reset_index(drop=True)

        pol = df.iloc[0,5]
        df = df.iloc[:,cols].rename(columns={df.columns[3]:'CDname',
                                             df.columns[4]:'CD',
                                             df.columns[6]:'{}-perc10'.format(pols[pol]),
                                             df.columns[7]:'{}-perc90'.format(pols[pol]),
                                             df.columns[8]:'{}-mean'.format(pols[pol])})

        new = df['Year'].str.split(" ", n=1, expand=True) 
        df['Year'] = new[1]
        df['Month'] = new[0]
        df['Day'] = 1

        for i in range(len(df)):
            if df.iloc[i,6] == 'Summer':
                df.iloc[i,6] = 5
            else:
                df.iloc[i,6] = 11

            df.iloc[i,1] = re.sub("[\(\[].*?[\)\]]", "", df.iloc[i,1])

            if '-' in df.iloc[i,0]:
                df.iloc[i,0] = df.iloc[i,0][:4]

        df2 = df[['Year','Month','Day']]
        df['Date'] = pd.to_datetime(df2)

        df = df.iloc[:,[8,1,2,3,4,5]]
        
        maindf = maindf.merge(df,on=['CDname','CD','Date'],how='outer')
        
    else:
        pass

In [10]:
maindf.head()

Date                         CDname   CD  BC-perc10  BC-perc90  \
0 2012-05-01   Bay Ridge and Dyker Heights   310        0.9        1.1   
1 2013-05-01   Bay Ridge and Dyker Heights   310        0.8        1.1   
2 2014-05-01   Bay Ridge and Dyker Heights   310        0.7        1.0   
3 2015-05-01   Bay Ridge and Dyker Heights   310        0.7        1.0   
4 2016-05-01   Bay Ridge and Dyker Heights   310        0.7        1.0   

   BC-mean  PM2.5-perc10_x  PM2.5-perc90_x  PM2.5-mean_x  PM2.5-perc10_y  ...  \
0      1.0             NaN             NaN           NaN             NaN  ...   
1      0.9             9.4            10.6           9.9             9.4  ...   
2      0.9             8.1             9.2           8.5             8.1  ...   
3      0.9             8.7             9.9           9.2             8.7  ...   
4      0.8             7.7             8.9           8.2             7.7  ...   

   O3-mean  NO-perc10  NO-perc90  NO-mean  SO2-perc10  SO2-perc90  SO2-mean  \
0     33.0        NaN        NaN      NaN         NaN         NaN       NaN   
1     30.3        6.1       13.7      9.2         NaN         NaN       NaN   
2     30.7        7.4       15.9     11.0         NaN         NaN       NaN   
3     31.8        6.7       14.6      9.8         NaN         NaN       NaN   
4     33.5        6.3        9.5      7.6         NaN         NaN       NaN   

   NO2-perc10  NO2-perc90  NO2-mean  
0         NaN         NaN       NaN  
1        11.0        17.9      15.2  
2        10.9        16.9      14.5  
3        10.7        16.5      14.2  
4        10.7        16.6      14.3  

[5 rows x 27 columns]

In [11]:
len(maindf.CD.unique())

59

In [12]:
len(maindf.Date.unique())

22

In [13]:
maindf.CD.unique()

array([310, 411, 303, 206, 311, 312, 316, 304, 110, 104, 313, 308, 317,
       111, 305, 404, 101, 314, 318, 407, 205, 302, 301, 102, 204, 408,
       202, 403, 412, 409, 207, 401, 103, 105, 109, 211, 203, 201, 306,
       209, 413, 406, 405, 208, 414, 315, 502, 309, 410, 501, 106, 307,
       210, 503, 108, 107, 112, 212, 402])

In [14]:
#maindf.to_csv('pollutants.csv')